In [1]:
import requests
from bs4 import BeautifulSoup

In [40]:
_BASE = 'https://doegen.ie/counties'
def do_get(url):
    r = requests.get(url, headers = {'User-agent': 'Mozilla/5.0'})
    if r.status_code != 200:
        raise Exception("Failed to open landing page")
    return r.content

In [10]:
soup = BeautifulSoup(do_get(_BASE), 'html.parser')

In [26]:
counties = soup.find('ul', {'class': 'vocabindex'}).find_all('li')

In [37]:
pages = []
for county in counties:
    item = {}
    anchor = county.find('a')
    href = anchor['href']
    item['link'] = f'https://doegen.ie{href}'
    if anchor.find('span').text.strip() != '(0)':
        item['county'] = anchor.text.split()[1]
        pages.append(item)

In [38]:
pages

[{'link': 'https://doegen.ie/taxonomy/term/22012', 'county': 'Antrim'},
 {'link': 'https://doegen.ie/taxonomy/term/22101', 'county': 'Armagh'},
 {'link': 'https://doegen.ie/taxonomy/term/22050', 'county': 'Cavan'},
 {'link': 'https://doegen.ie/taxonomy/term/21566', 'county': 'Clare'},
 {'link': 'https://doegen.ie/taxonomy/term/21510', 'county': 'Cork'},
 {'link': 'https://doegen.ie/taxonomy/term/22027', 'county': 'Derry'},
 {'link': 'https://doegen.ie/taxonomy/term/21979', 'county': 'Donegal'},
 {'link': 'https://doegen.ie/taxonomy/term/21995', 'county': 'Dublin'},
 {'link': 'https://doegen.ie/taxonomy/term/21520', 'county': 'Galway'},
 {'link': 'https://doegen.ie/taxonomy/term/21515', 'county': 'Kerry'},
 {'link': 'https://doegen.ie/taxonomy/term/21601', 'county': 'Leitrim'},
 {'link': 'https://doegen.ie/taxonomy/term/21984', 'county': 'Louth'},
 {'link': 'https://doegen.ie/taxonomy/term/21505', 'county': 'Mayo'},
 {'link': 'https://doegen.ie/taxonomy/term/21756', 'county': 'Roscommon

In [58]:
def proc_page(url):
    result = {}
    html = do_get(url)
    soup = BeautifulSoup(html, 'html.parser')
    content = soup.find('div', {'id': 'main'}).find('div', {'class': 'content'})
    result['transcript'] = content.find('div', id='tei_text')

In [51]:
def proc_county(item):
    content = do_get(item['link'])
    soup = BeautifulSoup(content, 'html.parser')
    main = soup.find('div', id='main')
    nodes = main.find_all('div', {'class': 'node'})
    stories = []
    for node in nodes:
        story = {}
        anchor = node.find('a')
        story['link'] = f"https://doegen.ie{anchor['href']}"
        text = anchor.text
        if ' - ' in text:
            tmp = text.split(' - ')
            if len(tmp) == 2:
                story['title'] = tmp[0]
                story['speaker'] = tmp[1]
            else:
                story['raw'] = text
        else:
            story['raw'] = text
        stories.append(story)
    item['stories'] = stories

In [52]:
tmp = {'link': 'https://doegen.ie/taxonomy/term/21528', 'county': 'Waterford'}
proc_county(tmp)
tmp

{'link': 'https://doegen.ie/taxonomy/term/21528',
 'county': 'Waterford',
 'stories': [{'link': 'https://doegen.ie/LA_1051d3',
   'title': 'An bhean phósta',
   'speaker': 'Mícheál Turraoin'},
  {'link': 'https://doegen.ie/LA_1051d1',
   'title': 'An garsún bán',
   'speaker': 'Mícheál Turraoin'},
  {'link': 'https://doegen.ie/LA_1052d1',
   'title': 'An mac scaiptheach (cuid 1)',
   'speaker': 'Tomás Ó Corcráin'},
  {'link': 'https://doegen.ie/LA_1052d2',
   'title': 'An mac scaiptheach (cuid 2)',
   'speaker': 'Tomás Ó Corcráin'},
  {'link': 'https://doegen.ie/LA_1053d1',
   'title': 'An mac scaiptheach (cuid 3)',
   'speaker': 'Tomás Ó Corcráin'},
  {'link': 'https://doegen.ie/LA_1045d1',
   'title': 'An mac scaiptheach',
   'speaker': 'Mícheál Ó Cionnfhaolaidh'},
  {'link': 'https://doegen.ie/LA_1050d1',
   'title': 'An mac scaiptheach',
   'speaker': 'Mícheál Turraoin'},
  {'link': 'https://doegen.ie/LA_1051d2',
   'title': 'Bean an Chárthaigh',
   'speaker': 'Mícheál Turraoin'},


In [59]:
proc_page('https://doegen.ie/LA_1051d3')

<div class="content">
<audio controls="" height="27" width="320"><source src="http://assets.doegen.ie/sound/MP3_versions/aud_Mu3-LA_1051d3u1.mp3" type="audio/mp3"/><source src="http://assets.doegen.ie/sound/ogg_versions/aud_Mu3-LA_1051d3u1.ogg" type="audio/ogg"/><embed height="27" src="http://assets.doegen.ie/sound/MP3_versions/aud_Mu3-LA_1051d3u1.mp3" width="320"/><br/><a href="http://assets.doegen.ie/sound/MP3_versions/aud_Mu3-LA_1051d3u1.mp3">Play in separate window.</a></audio><br/>
<p><b>Recording:</b> <span id="mp3download">[<a href="http://www.doegen.ie/fodl.php?id=doegen0:aud_Mu3-LA_1051d3u1&amp;nm=aud_Mu3-LA_1051d3u1&amp;ds=LoDef">Download audio file</a>]</span>
<span id="aiffdownload">[<a href="http://www.doegen.ie/fodl.php?id=doegen0:aud_Mu3-LA_1051d3u1&amp;nm=aud_Mu3-LA_1051d3u1&amp;ds=HiDef">Download AIFF audio file (of processed ‘user’ version)</a>]</span>
<span id="aiffarchivedownload">[<a href="http://www.doegen.ie/fodl.php?id=doegen0:aud_Mu3-LA_1051d3a1&amp;nm=aud_Mu3-

In [45]:
for node in main.find_all('div', {'class': 'node'}):
    print(node.find('a'))

<a href="/LA_1051d3" title="An bhean phósta - Mícheál Turraoin">An bhean phósta - Mícheál Turraoin</a>
<a href="/LA_1051d1" title="An garsún bán - Mícheál Turraoin">An garsún bán - Mícheál Turraoin</a>
<a href="/LA_1052d1" title="An mac scaiptheach (cuid 1) - Tomás Ó Corcráin">An mac scaiptheach (cuid 1) - Tomás Ó Corcráin</a>
<a href="/LA_1052d2" title="An mac scaiptheach (cuid 2) - Tomás Ó Corcráin">An mac scaiptheach (cuid 2) - Tomás Ó Corcráin</a>
<a href="/LA_1053d1" title="An mac scaiptheach (cuid 3) - Tomás Ó Corcráin">An mac scaiptheach (cuid 3) - Tomás Ó Corcráin</a>
<a href="/LA_1045d1" title="An mac scaiptheach - Mícheál Ó Cionnfhaolaidh">An mac scaiptheach - Mícheál Ó Cionnfhaolaidh</a>
<a href="/LA_1050d1" title="An mac scaiptheach - Mícheál Turraoin">An mac scaiptheach - Mícheál Turraoin</a>
<a href="/LA_1051d2" title="Bean an Chárthaigh - Mícheál Turraoin">Bean an Chárthaigh - Mícheál Turraoin</a>
<a href="/LA_1051d4" title="Bean an doichill - Mícheál Turraoin">Bean an d